In [2]:
#Analisis Covid 19

In [3]:
#Configuración del ambiente 
import pandas as pd
import numpy as np 
import requests
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from sklearn.ensemble import RandomForestRegressor
import json

In [4]:
df_covid = pd.read_csv(StringIO(requests.get("https://covid19.who.int/WHO-COVID-19-global-data.csv").text))
df_population = pd.read_excel('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx', sheet_name=0, skiprows=16)
df_population = df_population[df_population['Year'] == 2019]

In [5]:
df_covid

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,EMRO,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,EMRO,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,EMRO,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,EMRO,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,EMRO,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...
50635,2023-12-17,ZW,Zimbabwe,AFRO,50.0,266025,NaN,5730
50636,2023-12-24,ZW,Zimbabwe,AFRO,23.0,266048,1.0,5731
50637,2023-12-31,ZW,Zimbabwe,AFRO,-119.0,265929,-7.0,5724
50638,2024-01-07,ZW,Zimbabwe,AFRO,46.0,265975,3.0,5727


In [6]:
# Hallar nulos
df_covid.isna().sum().sort_values(ascending = False)

New_deaths           26479
New_cases            14112
WHO_region            5064
Country_code          1477
Country               1266
Date_reported            0
Cumulative_cases         0
Cumulative_deaths        0
dtype: int64

In [7]:
#Eliminar todos los registros con nulos en la columna Country_code.
df_covid = df_covid.dropna(subset=['Country_code'])
df_covid

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,EMRO,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,EMRO,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,EMRO,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,EMRO,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,EMRO,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...
50635,2023-12-17,ZW,Zimbabwe,AFRO,50.0,266025,NaN,5730
50636,2023-12-24,ZW,Zimbabwe,AFRO,23.0,266048,1.0,5731
50637,2023-12-31,ZW,Zimbabwe,AFRO,-119.0,265929,-7.0,5724
50638,2024-01-07,ZW,Zimbabwe,AFRO,46.0,265975,3.0,5727


In [8]:
#Mantener en el dataframe sólo las siguientes columnas:
df_covid = df_covid[['Date_reported','Country_code','Country','New_cases','Cumulative_cases','New_deaths','Cumulative_deaths']]
df_covid

,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,NaN,0,NaN,0
...,...,...,...,...,...,...,...
50635,2023-12-17,ZW,Zimbabwe,50.0,266025,NaN,5730
50636,2023-12-24,ZW,Zimbabwe,23.0,266048,1.0,5731
50637,2023-12-31,ZW,Zimbabwe,-119.0,265929,-7.0,5724
50638,2024-01-07,ZW,Zimbabwe,46.0,265975,3.0,5727


In [9]:
#Sustituir todos los valores nulos con cero.
df_covid.fillna(0, inplace=True)
df_covid

C:\Users\Edgar Fragoso\AppData\Local\Temp\ipykernel_25144\1875090180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid.fillna(0, inplace=True)


,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,0.0,0,0.0,0
1,2020-01-12,AF,Afghanistan,0.0,0,0.0,0
2,2020-01-19,AF,Afghanistan,0.0,0,0.0,0
3,2020-01-26,AF,Afghanistan,0.0,0,0.0,0
4,2020-02-02,AF,Afghanistan,0.0,0,0.0,0
...,...,...,...,...,...,...,...
50635,2023-12-17,ZW,Zimbabwe,50.0,266025,0.0,5730
50636,2023-12-24,ZW,Zimbabwe,23.0,266048,1.0,5731
50637,2023-12-31,ZW,Zimbabwe,-119.0,265929,-7.0,5724
50638,2024-01-07,ZW,Zimbabwe,46.0,265975,3.0,5727


In [10]:
#Cambiar el formato actual de la columna Date_reported para un formato de tipo datetime64[ns].
df_covid["Date_reported"] = pd.to_datetime(df_covid["Date_reported"])


C:\Users\Edgar Fragoso\AppData\Local\Temp\ipykernel_25144\1474962662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid["Date_reported"] = pd.to_datetime(df_covid["Date_reported"])


In [11]:
#Substituir el formato actual de las columnas ['New_cases', 'New_deaths']para un formato de tipo int64.
df_covid["New_deaths"] = pd.to_numeric(df_covid["New_deaths"].astype('int64'))
df_covid["New_cases"] = pd.to_numeric(df_covid["New_cases"].astype('int64'))


C:\Users\Edgar Fragoso\AppData\Local\Temp\ipykernel_25144\2467875683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid["New_deaths"] = pd.to_numeric(df_covid["New_deaths"].astype('int64'))
C:\Users\Edgar Fragoso\AppData\Local\Temp\ipykernel_25144\2467875683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid["New_cases"] = pd.to_numeric(df_covid["New_cases"].astype('int64'))


In [12]:
#Crear una nueva columna 'lethality_rate' con la siguiente fórmula: 'Cumulative_deaths'/'Cumulative_cases'*100
df_covid['lethality_rate'] = df_covid['Cumulative_deaths'] / df_covid['Cumulative_cases'] * 100
df_covid

,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,lethality_rate
0,2020-01-05,AF,Afghanistan,0,0,0,0,NaN
1,2020-01-12,AF,Afghanistan,0,0,0,0,NaN
2,2020-01-19,AF,Afghanistan,0,0,0,0,NaN
3,2020-01-26,AF,Afghanistan,0,0,0,0,NaN
4,2020-02-02,AF,Afghanistan,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...
50635,2023-12-17,ZW,Zimbabwe,50,266025,0,5730,2.153933
50636,2023-12-24,ZW,Zimbabwe,23,266048,1,5731,2.154123
50637,2023-12-31,ZW,Zimbabwe,-119,265929,-7,5724,2.152454
50638,2024-01-07,ZW,Zimbabwe,46,265975,3,5727,2.153210


In [13]:
#Con el fin de eliminar outliers, nos quedaremos con los registros cuyo valor en la columna lethality_rate se encuentre entre los cuantiles 0 a 0.99.
df_covid = df_covid[(df_covid['lethality_rate'] > 0) & (df_covid['lethality_rate'] < 0.99)]
df_covid

,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,lethality_rate
749,2022-03-27,AS,American Samoa,1881,3278,2,2,0.061013
750,2022-04-03,AS,American Samoa,1378,4656,5,7,0.150344
751,2022-04-10,AS,American Samoa,746,5402,4,11,0.203628
752,2022-04-17,AS,American Samoa,133,5535,7,18,0.325203
753,2022-04-24,AS,American Samoa,230,5765,5,23,0.398959
...,...,...,...,...,...,...,...,...
50238,2020-05-24,ZM,Zambia,241,920,0,9,0.978261
50239,2020-05-31,ZM,Zambia,137,1057,0,9,0.851466
50240,2020-06-07,ZM,Zambia,97,1154,0,9,0.779896
50241,2020-06-14,ZM,Zambia,203,1357,0,9,0.663228


In [14]:
#Finalmente reiniciaremos el índice del dataframe final df_covid_limpio
df_covid_limpio = df_covid.reset_index(drop=True)
df_covid_limpio

,Date_reported,Country_code,Country,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,lethality_rate
0,2022-03-27,AS,American Samoa,1881,3278,2,2,0.061013
1,2022-04-03,AS,American Samoa,1378,4656,5,7,0.150344
2,2022-04-10,AS,American Samoa,746,5402,4,11,0.203628
3,2022-04-17,AS,American Samoa,133,5535,7,18,0.325203
4,2022-04-24,AS,American Samoa,230,5765,5,23,0.398959
...,...,...,...,...,...,...,...,...
16745,2020-05-24,ZM,Zambia,241,920,0,9,0.978261
16746,2020-05-31,ZM,Zambia,137,1057,0,9,0.851466
16747,2020-06-07,ZM,Zambia,97,1154,0,9,0.779896
16748,2020-06-14,ZM,Zambia,203,1357,0,9,0.663228


In [15]:
#Corroboramos cambios y que la limpieza ha sido efectuada
df_covid_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16750 entries, 0 to 16749
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date_reported      16750 non-null  datetime64[ns]
 1   Country_code       16750 non-null  object        
 2   Country            16750 non-null  object        
 3   New_cases          16750 non-null  int64         
 4   Cumulative_cases   16750 non-null  int64         
 5   New_deaths         16750 non-null  int64         
 6   Cumulative_deaths  16750 non-null  int64         
 7   lethality_rate     16750 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 1.0+ MB


In [16]:
#Almacenar el dataFrame en un csv para trabajar las etapas por separado.
df_covid_limpio.to_csv('df_covid_limpio.csv', sep=',', index=False)

In [20]:
#Observamos el contenido del dataset
df_population

,ISO2 Alpha-code,"Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Life Expectancy at Birth, both sexes (years)"
69,NaN,7764951.032,3906407.855,3858543.178,59.546,72.79
142,NaN,1080677.346,538158.386,542518.959,49.425,60.725
214,NaN,530149.56,273104.233,257045.327,42.485,73.392
286,NaN,2021559.651,1033583.865,987975.786,195.683,70.615
358,NaN,2324282.002,1176368.32,1147913.681,143.117,77.013
...,...,...,...,...,...,...
20305,WS,211.905,108.147,103.759,74.878,72.157
20377,TK,1.752,0.879,0.874,146,75.076
20449,TO,104.951,51.994,52.956,161.462,70.871
20521,TV,10.956,5.654,5.301,365.2,64.278


In [21]:
#Mantener en el dataframe sólo las siguientes columnas: ['ISO2 Alpha-code','Total Population, as of 1 July (thousands)','Male Population, as of 1 July (thousands)','Female Population, as of 1 July (thousands)','Population Density, as of 1 July (persons per square km)','Life Expectancy at Birth, both sexes (years)'].
df_population = df_population[['ISO2 Alpha-code','Total Population, as of 1 July (thousands)','Male Population, as of 1 July (thousands)','Female Population, as of 1 July (thousands)','Population Density, as of 1 July (persons per square km)','Life Expectancy at Birth, both sexes (years)']]
df_population

,ISO2 Alpha-code,"Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Life Expectancy at Birth, both sexes (years)"
69,NaN,7764951.032,3906407.855,3858543.178,59.546,72.79
142,NaN,1080677.346,538158.386,542518.959,49.425,60.725
214,NaN,530149.56,273104.233,257045.327,42.485,73.392
286,NaN,2021559.651,1033583.865,987975.786,195.683,70.615
358,NaN,2324282.002,1176368.32,1147913.681,143.117,77.013
...,...,...,...,...,...,...
20305,WS,211.905,108.147,103.759,74.878,72.157
20377,TK,1.752,0.879,0.874,146,75.076
20449,TO,104.951,51.994,52.956,161.462,70.871
20521,TV,10.956,5.654,5.301,365.2,64.278


In [ ]:
#Renombrar las columnas para nombre más sencillos:
df_population = df_population.rename(columns={
    'ISO2 Alpha-code': 'Country_code',
    'Total Population, as of 1 July (thousands)': 'Total_Population',
    'Male Population, as of 1 July (thousands)': 'Male_Population',
    'Female Population, as of 1 July (thousands)': 'Female_Population',
    'Population Density, as of 1 July (persons per square km)': 'Population_Density',
    'Life Expectancy at Birth, both sexes (years)': 'Life_Expectancy'
})
df_population

In [ ]:
#Eliminar todos los registros nulos.
# Hallar nulos
df_population.isna().sum().sort_values(ascending = False)

In [ ]:
# Borrar registros nulos
df_population = df_population.dropna()
df_population

In [ ]:
#Substituir el formato actual de las columnas ['Total_Population','Male_Population','Female_Population','Population_Density','Life_Expectancy']para un formato de tipo int64.
df_population.info()

In [ ]:
# Encontrar una fila con una condicion
n = df_population[df_population['Male_Population'] == '...']
n

In [ ]:
# Borrar teniendo el indice
df_population.drop(df_population[df_population['Male_Population'] == '...'].index, inplace=True)

In [ ]:
df_population

In [ ]:
#Multiplicar por 1000 el valor actual de las siguientes columnas 'Total_Population','Male_Population','Female_Population' y sobreescribirlas.
df_population[['Total_Population','Male_Population','Female_Population']] = df_population[['Total_Population','Male_Population','Female_Population']] * 1000
df_population

In [ ]:
#Volver a eliminar todos los registros nulos.
# Buscar nulos
df_population.isna().sum().sort_values(ascending = False)

In [ ]:
#Finalmente reiniciaremos el índice del dataframe final df_population_limpio
# Reiniciar índice
df_population_limpio = df_population.reset_index(drop=True)
df_population_limpio

In [ ]:
df_population_limpio.info()

In [ ]:
df_covid_limpio.info()

In [ ]:
# Celda 1
datos = {"clave": "valor"}
with open("datos.json", "w") as archivo:
    json.dump(datos, archivo)

# Celda 2 en otra pestaña/notebook
with open("datos.json", "r") as archivo:
    datos_cargados = json.load(archivo)
print(datos_cargados)  # Salida: {'clave': 'valor'}
